# Text Preprocessing für NLP

## NLP (Natural Language Processing)

Natural language processing (NLP) ist ein Zweig der Informatik, oder genauer gesagt der künstlichen Intelligenz, der versucht, geschriebene und gesprochene Sprache für den Computer "verstehbar" zu machen.

### Einige Aufgaben von NLP:

* Automatische Übersetzung zwischen Sprachen
* Spracherkennung 
* Autmatisches Beantworten von Fragen
* Zusammenfassung von Texten
* Sentiment Analysis: Stimmungsanalyse
* Topic Modelling: thematische Exploration von Texten
* Named Entity Recognition (NER): Erkennen von Personen, Orten, Firmen usw. in fortlaufenden Texten
* Speech to text and text to speech: Gesprochende Texte in geschriebene Texte umwandeln und vice versa.
* Automatische Erzeugung von Texten und Fragen (text and question generation)
* Rechtschreibkorrektur/Fehlerkorrektur
* Disambiguierung (word-sense disambiguation): "Verstehen" der korrekten Bedeutung von mehrdeutigen Wörtern und Ausdrücken

## Was ist Preprocessing von Texten?

Natürlichsprachige Texte befinden sich meist nicht in einem Zustand, der für die computergestützte Verarbeitung ideal ist. Daher müssen wir die Daten zuerst in einen Zustand bringen, der die eigentliche Analyse erleichtert, verbessert oder überhaupt erst möglich macht. Erfahrungsgemäß tendieren wir dazu, diesem Schritt nicht ausreichend Aufmerksamkeit zu widmen, was sich dann später im Analyseprozess rächt. Deshalb sehen wir uns hier typische Aufgaben des Preprocessings an.

Für Computer ist das Verstehen natürlicher Sprache eine schwierige Aufgabe, weil die Daten nicht in strukturierter Form (wie etwa in einer Tabelle oder relationalen Datenbank) vorliegen. Diese Strings (z. B. Tweets, Artikel, Rezensionen, Romane, Gedichte, Theaterstücke) oder Sprachaufnahmen sind also in der Regel unstrukturiert. Durch die Vorverarbeitung wird der Text in eine für den Computer besser verdauliche Form gebracht, so dass unsere NLP-Methoden (z. B. auf der Grundlage von Statistik oder von Algorithmen des maschinellen Lernens) besser funktionieren.

Eine erste Hürde stellt bereits das Problem dar, herauszufinden, in welcher Hinsicht unsere Daten "unordentlich" sind, damit wir diese sinnvoll bereinigen können. Dieser Schritt ist mühsam und oft langweilig. Dennoch dürfen wir nicht darauf verzichten: Wenn unsere Daten nicht sauber sind, werden auch die Ergebnisse unserer Analyse nicht sauber sein.

Die Auswahl, Umsetzung und Kombination von Vorverarbeitungsschritten hängt stark von der Domäne und der zu untersuchenden Fragestellung ab. Daher müssen wir nicht alle Schritte auf jedes Problem anwenden. Oft braucht es mehrere Iterationszyklen (Preprocessing -> Processing -> Preprocessing -> Processing etc.), wenn wir bei der Analyse fehlende Bereinigungsschritte erkennen.

In diesem Notebook werden einige der üblichen Vorverarbeitungsschritte behandelt. Abhängig von Ihren Daten und Ihrer Aufgabe (sowie der NLP-Methode, die Sie verwenden möchten) werden Sie nur einige oder vielleicht sogar alle Schritte in diesem Leitfaden benötigen.

### Häufige Preprocessing Schritte

* Umwandlung in Kleinbuchstaben (vereinheitlichte Schreibweise)
* Tokenization (Texte in kleinere "Tokens" zerlegen)
* Satzzeichen entfernen
* URLs, Tags oder andere Steuerzeichen entfernen
* Stopwords entfernen
* Stemming (Wortstamm extrahieren)
* Lemmatisierung (Flexionsformen auf Grundform bringen)

## Preprocessing in der Praxis

Für viele der folgenden Schritte gibt es fertige Funktionen in diversen Bibliotheken. Um ein tieferes Verständnis davon zu erzielen, was wir da eigentlich tun, demonstriere ich zunächst einmal ein paar typische Preprocessing-Schritte mit Mitteln der Standard Library. 

Natürlich sollten wie das Rad nicht immer neu erfinden, sondern in der praktischen Arbeit Bibliotheken einsetzen, wie beispielhaft im zweiten Teil beschrieben. Der Vorteil ist nicht nur die Zeitersparnis, sondern auch, dass die von Biblotheken bereit gestellten Funktionen in der Regel leistungsfähiger, optimierter und besser getestet sind.

Zunächst importieren wir zwei Bibliotheken aus der Standard Library, die den Umgang mit Texten erleichtern:

* string (https://docs.python.org/library/string.html) bietet für unsere Zwecke vor allem eine Reihe von nützlichen Konstanten wie etwa ``ascii_letters``, ``ascii_lowercase`` oder ``punctuation``.
* re ist die Regex-Bibliothek aus der Standard Library (zu Regulären Ausdrücken und dem re-Modul gibt es ein eigenes Notebook).

In [ ]:
import string
import re

### Text einlesen

In [ ]:
with open('data/cactus.txt') as fh:
    text = fh.read()
text

### Zeilenumbrüche entfernen

Zum Entfernen der Zeilenumbrüche gibt es mehrere Möglichkeiten, wir verwenden hier die replace Methode des String Objekts.

In [ ]:
text = text.replace("\n", " ")
text

Eine andere, vermutlich bessere Möglichkeit (weil diese gleich auch mehrfach vorkommenden Whitespace auf ein Leerzeichen reduziert) wäre die ``sub`` Methode von re:

In [ ]:
# We read the text from file again, because we aleady
# have modified text in the cell above
with open('data/cactus.txt') as fh:
    text = fh.read()

# replace any number of subsequent whitespace to a single space
text = re.sub(r'\s+', ' ', text)
text[:140]

### Alles in Groß- oder Kleinbuchstaben umwandeln

Im Normalfall wollen wir diese Umwandlung, weil die Wörter dieselben sind, auch wenn sie am Satzanfang groß geschrieben werden. Wenn wir die Groß/Kleinschreibung vereinheitlichen, müssen wir später bei der Analyse keine eigenen Regeln dafür bilden (das Vokabular unserer Textdaten wird reduziert). Bei manchen Sprachen wie Deutsch ist es allerdings so, dass die Beibehaltung der Groß- und Kleinschreibung die Ergebnisse des Part-of-Speech-Taggings verbessern kann. Im Zweifelsfall lohnt es sich, die Ergebnisse zu vergleichen.

Die dazu benötigten Methoden ``lower()`` und  ``upper()`` stehen als String-Methoden zur Verfügung.

In [ ]:
lower_text = text.lower()
lower_text[:140]

### Tokenization

Als Tokenization bezeichnet man den Schritt, in dem ein Text in kleinere Einheiten wie z.B. Wörter oder Sätze zerlegt wird.

Dazu können wir die ``split()`` Methode des String-Objekts verwenden. Diese Methode trennt einen String an jedem Vorkommen eines Substrings auf und liefert die einzelnen Teile als Liste. Im einfachsten Falle rufen wìr ``split()`` ohne Argument auf. Dabei verwendet ``split()`` Whitespace-Zeichen (d.h. Leerzeichen, Tabulatoren, Zeilenumbrüche usw.) als Trenner:

In [ ]:
tokens = lower_text.split() 
tokens[:10]

Wollen wir statt dessen in Sätze zerlegen wollen, können wir bei ``split()`` den Punkt als Trennzeichen angeben.

In [ ]:
sentences = lower_text.split('.')
sentences[:10]

Wir sehen hier allerdings, dass das Ergebnis teilweise unerwünschte Leezzeichen enthält. Die rühren daher, dass nach einem Punkt normalerweise ein Leerzeichen folgt. Das Problem könnten wir damit lösen,
dass wir an ``'. '``  (Punkt Leerzeichen) splitten. Das birgt jedoch die Gefahr, dass das Leezeichen nach dem Punkt nicht immer vorhanden sein muss (etwa am Ende des Textes).

In [ ]:
sentences = lower_text.split('. ')
sentences[:10]

 Kehren wir also zur ersten Lösung zurück und entfernen die störenden Leezeichen mit der ``strip()`` Methode in einer List Comprehension:

In [ ]:
sentences = lower_text.split('.')
stripped = [s.strip() for s in sentences]
stripped[-10:]

Bei genauerer Betrachtung sehen wir möglicherweise noch, dass es irgendwo leere Tokens gibt, hier am Ende der Liste. Da diese für unsere Analyse keinerlei Bedeutung haben, sollten wir sie ebenefalls entfernen:

In [ ]:
new_list = [x for x in stripped if x != '']
new_list[-10:]

Eine Alternative besteht natürlich in der Anwendung Regulärer Ausdrücke. Hier splitten wir an Punkt, Fragezeichen und Rufzeichen, wobei beliebige viele Leerzeichen nach den Trennzeichen (die oben zum leeren Token geführt haben) ignoriert werden.

In [ ]:
sentences = [sentence for sentence in re.split(r'[.?!]\s*', lower_text) if sentence]
sentences[-10:]

## Preprocessing mit NLTK

### NLTK

NLTK das *Natural Language Toolkit* (https://www.nltk.org/) ist eine populäre Python Bibliothek zur Verarbeitung natürlicher Sprache.
Mit der folgenden Code-Zelle können Sie testen, ob ntlk auf Ihrem Computer bereits installiert ist:

In [ ]:
import nltk

Falls Sie bei der Ausführung der Zelle eine Fehlermeldung erhalten haben, müssen Sie Sie es mit

```
pip install nltk
```

bzw.

```
conda install nltk 
```

installieren. Grundsätzlich sollte das auch aus diesem Notebook funktionieren. Entfernen Sie dazu das Kommentarzeichen (#) in einer der beiden folgenden Zeilen und führen Sie dann die Zelle aus.

In [ ]:
#!pip install nltk
#!conda install nltk

### Tokenizers

Der oben beschriebene Ansatz, der sich auf Möglichkeiten der Standard Library beschränkt, empfiehlt sich nur für sehr einfache Anwendungsfälle. In der Praxis gibt es eine Reihe von Schwierigkeiten (z.B. Punkte als Dezimaltrenner, weitere Satzzeichen wie ! oder ?, dazu möglicherweise unerwartete Satzzeichen wie das spanische ``¿``, die die Aufgabe erschweren. Daher sollte man spezialisierte Bibliotheken verwenden. 

Das NLTK Package etwa bietet mehrere Tokenizer-Funktionen wie ``sent_tokenize()`` für Sätze oder ``word_tokenize()`` für Wörter, wo diese Spezialfälle zumindest zum Teil bereits mitgedacht sind. Damit das funktioniert, müssen wir (einmal) den``punkt`` Tokenizer des NLTK heruntladen.

In [ ]:
nltk.download('punkt')  # in case it has not been installed yet

#### sent_tokenize

Im nächsten Schritt verwenden wir die ``sent_tokenize()`` Funktion von nltk, um den Text in einzelne Sätze zu zerlegen.

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
sentences = sent_tokenize(lower_text)
sentences

Bei ``sent_tokenize()`` (wie bei allen `punkt` Tokenizern ist zu beachten, dass die Satzzeichen bewahrt werden. Falls wir an den Satzzeichen nicht interessiert sind, sollten wir diese in einem weiteren Schritt entfernen. Dabei machen wir uns zunutze, dass das string Modul der Standard Library die Satzzeichen als Konstante (``string.punctuation``) vordefiniert hat.

In [ ]:
stripped_sentences = []
for sentence in w_sentences:
    if sentence[-1] in string.punctuation:
        stripped_sentences.append(sentence[:-1].strip())
    else:
        stripped_sentences.append(sentence)
stripped_sentences[:3]

##### Sprachspezifische Tokenizer
Die Tokenizer-Funktionen können auch an eine bestimmte Sprache (default ist Englisch) angepasst werden. Im folgenden Beispiel tokenisieren wir einen deutschsprachigen Roman von Jakob Wassermann: 

In [ ]:
with open('data/wassermann/der_mann_von_vierzig_jahren.txt', encoding='utf-8') as fh:
    w_text = re.sub(r'\s+', ' ', fh.read().lower())
w_sentences = sent_tokenize(w_text, language='german')
w_sentences[:3]

#### word_tokenize

Der Word-Tokenizer liefert eine Liste von Wörtern. Im folgenden Beispiel zerlegen wir den Text mit Hilfe der NLTK ``word_tokenize()``Funktion in einzelne Wörter:

In [ ]:
words = word_tokenize(lower_text)
words[:11]

Wir sehen, dass auch hier die Satzzeichen als Tokens erhalten bleiben.
Je nach Fragestellung kann das nützlich sein oder auch nicht. Falls wir an den Satzzeichen nicht interessiert sind, können wir diese in einer List Comprehension entfernen. Auch hier machen wir uns zunutze, dass im ``string`` Modul die Satzzeichen als Konstante definiert sind.

In [ ]:
words = [token for token in words if token not in string.punctuation]
words[:11]

### Stopwords entfernen

Stopwords sind eine Liste von Wörtern, an denen wir nicht interessiert sind. Das sind z.B. Füllwörter, Artikel und Pronomen. Wie können diese Listen selbst erstellen, was allerdings relativ mühsam ist. Besser sucht man sich eine im Internet bereit gestellte Stopwortliste für die entsprechende Sprache oder man verwendet z.B. eine der von NLTK bereit gestellte Liste.

In [ ]:
nltk.download('stopwords')  # only needed once!

In [ ]:
from nltk.corpus import stopwords
german_stopwords = stopwords.words('german')
print(f"Die ersten 10 Stopwords: {german_stopwords[:10]}")
print(f"Insgesamt sind {len(german_stopwords)} Stopwords definiert.")

In der Praxis nimmt man häufig eine solche bestehende Stopwortliste und passt sie an die eigenen Daten an, indem man weitere Stopwords hinzufügt. In unserem Beispiel verwenden wir eine solche individuelle Stopwortliste aus der Datei ``stopwords.txt`` im ``data`` Verzeichnis. Diese müssen wir natürlich einlesen und die einzelnen Stopwords als Liste bereitstellen:

In [ ]:
with open('data/stopwords.txt') as fp:
    stopwords = [word.rstrip() for word in fp.readlines()]
stopwords[:10]    

Dann verwenden wir wieder eine List Comprehension, um alle Stopwords aus unserer Tokens-Liste zu entfernen. Zur Kontrolle lassen wir uns die Zahl der Tokens vor und nach diesem Schritt ausgeben:

In [ ]:
print(len(words)) 
tokens = [token for token in words if token not in stopwords]
len(tokens)

Wir haben also 70 Stopwords entfernt, die unsere anstehende Auswertung verwässert hätten. Das sind immerhin fast 50% der Tokens!

### Stemming

Stemming nennt sich der Prozess, Wortformen auf ihren Wortstamm zu reduzieren. Dabei werden Präfixe und Suffixe entfernt. Da diese regelbasiert funktioniert, ist das Ergebnis nicht immer korrekt. Es ist aber in der Regel besser, mit diesen unvollkommenen Bereinigungen zu arbeiten als mit unbereinigten Daten.

Beispiele:

```
books      --->    book
looked     --->    look
denied     --->    deni
flies      --->    fli
```


In [ ]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()
stems = [stemmer.stem(word) for word in tokens]
# print the original token and the stammatized version
for i, token in enumerate(tokens):
    print(f"{token} -> {stems[i]}")

### Lemmatization

Lemmatisierung vereinheitlicht ebenfalls Wortformen auf ihre Grundform. Lemmatisierung ist aber ungleich mächtiger (und komplizierter) als Stemming, weil es eine morphologische Analyse durchführt. Es liefert das Lemma, also die Grundform aller Flexionsformen eines Wortes und berücksichtigt dabei Wissen über eine Sprache. Das geht so weit, dass sogar unregelmässige Formen wie das lateinische 
*tollere, sustulī, sublatum* korrekt behandelt werden. Lemmatisierung liefert also immer eine gültigen Wortform.

Beispiele:

```
books      --->    book
looked     --->    look
denied     --->    deny
flies      --->    fly
```


NLTK stellt den WordNetLemmatizer bereit, über den wir Lemmatisierungen vornehmen können:

In [ ]:
# Install data if not installed yet
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')  # install data for POS

In [ ]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [ ]:
lemmatizer = WordNetLemmatizer()

Damit die Lemmatisierung funktioniert, benötigen wir auch noch Part-Of-Speech Tagging (POS). Die ist nötig, weil die Lemmatisierung den grammatischen Kontext des Wortes benötigt. 

Ein POS-Tagger ergänzt das Token mit einem so genannten POS Tag, der die Funktion des Wortes festlegt ('ADJ', 'NOUN', usw.). Wir haben also nach dem POS-Tagging ein Tupel, das beispielsweise so aussieht: ``('water', 'NN')``. Leider verwenden unterschiedliche Tagger unterschiedliche und auch unterschiedlich differenzierte Tagsets. Die Bezeichnungen der Tags können sich also von Tagger zu Tagger unterscheiden. Der NLTK-Tagger liefert z.B. den String 'NN' für ein Nomen. 

Der WordNetLemmatizer erwartet hier aber einen Wert der sich (intern) hinter der Konstante ``wordnet.NOUN`` verbirgt. Deshalb müssen wir die Tag-Werte übersetzen, ehe wir sie an den Lemmatizer übergeben. Dazu verwenden wir die einfache Funktion
`get_wordnet_pos()`. Diese ersetzt z.B. alle Tags, die mit `N` beginnen (daher auch ``NN``) durch den Wert von wordnet.NOUN.

In [ ]:
# This is a helper function to map NTLK position tags 
# to wordnet tags
# Full list is available here: https://www.ling.upenn.edu/courses/Fall_2003/ling001/penn_treebank_pos.html
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

Führen wir zunächst das Part-of-Speech (POS) Tagging durch und lassen und das Ergebnis ausgeben:

In [ ]:
# Get tags
word_pos_tags = nltk.pos_tag(tokens)
word_pos_tags

Die Liste ``word_pos_tags`` enthält also zweiwertige Tupel: An Position 0 das Token und an Position 1 den ermittelten Position Tag (also die grammatische Funktion des Tokens im Satz:
Verb, Noun, Adjective, Adverbe usw.)

Im nächsten Schritt führen wir für jedes Token zwei Schritte durch:

  1) Wir übersetzen den vom POS-Tagger gelieferten Position Tag für das 
     jeweilige Token in die Form, die der Wordnet Lemmatizer erwartet: 
     ``get_wordnet_pos(tag[1]``). Aus ``NNS`` wird dabei beispielsweise der 
     Wert der Konstante ``wordnet.NOUN``.
  2) Wir übergeben das Token zusammen mit dem in Schritt 1 ermittelten Position 
     Tag an die ``lemmatize()`` Methode des Lemmatizers, die uns die lemmatisierte 
     Form des Tokens liefert. Das Ergebnis, d.h. die lemmatisierten Tokens speichern wir in die Liste
     ``lemmatized_words``.

In [ ]:
lemmatized_words = []
# Map the position tag and lemmatize the word/token
for i, tag in enumerate(word_pos_tags):
    lemmatized_words.append(lemmatizer.lemmatize(tag[0], get_wordnet_pos(tag[1])))

Zur Kontrolle können wir uns die Token und deren lemmatisierte Form zusammen ausgeben lassen. Das Ergebnis wirkt auf den ersten Blick etwas enttäuschend, weil die meisten Tokens im Text bereits in lemmatisierter Form erscheinen. Betrachten Sie aber beispielsweise ``looks``, ``growing``, ``tried`` oder ``asked``.

In [ ]:
for i, token in enumerate(word_pos_tags):
    print(f"{token[0]} -> {lemmatized_words[i]}")

Wir können und die bereinigten und lemmatisierten Tokens auch wieder als Text ausgeben lassen. Dies ist der Text, den wir sinnvoll für diverse Analysen verwenden können.

In [ ]:
# the lemmatize() method takes in the 
lemmatized_text = " ".join(lemmatized_words)
lemmatized_text

<div class="alert alert-block alert-info">
<b>Übung zum Preprocessing</b>
<p>Preprozessieren Sie den Inhalt der Datei 'data/cat.txt'. Das Ergebnis sollte ein sauberer lemmatisierter Text sein.</p>
</div>